In [8]:
from data import *
from pretrained_models import *
from torch.utils.data import DataLoader

In [5]:
MediaPipeCFG = MediaPipeCfg("pretrained_model/hand_landmarker.task")
options = MediaPipeCFG.create_options()
MP_model = MediaPipeCFG.HandLandmarker.create_from_options(options)

In [6]:
MMPoseCFG = MMPoseCfg(checkpoint_path='pretrained_model/checkpoint/rtmpose-s_simcc-body7_pt-body7_420e-256x192-acd4a1ef_20230504.pth',
                      config_path='pretrained_model/mmpose_config/rtmpose_m_8xb256-420e_coco-256x192.py')
body_model = MMPoseCFG.create_model()

Loads checkpoint by local backend from path: pretrained_model/checkpoint/rtmpose-s_simcc-body7_pt-body7_420e-256x192-acd4a1ef_20230504.pth
The model and loaded state dict do not match exactly

size mismatch for backbone.stem.0.conv.weight: copying a param with shape torch.Size([16, 3, 3, 3]) from checkpoint, the shape in current model is torch.Size([24, 3, 3, 3]).
size mismatch for backbone.stem.0.bn.weight: copying a param with shape torch.Size([16]) from checkpoint, the shape in current model is torch.Size([24]).
size mismatch for backbone.stem.0.bn.bias: copying a param with shape torch.Size([16]) from checkpoint, the shape in current model is torch.Size([24]).
size mismatch for backbone.stem.0.bn.running_mean: copying a param with shape torch.Size([16]) from checkpoint, the shape in current model is torch.Size([24]).
size mismatch for backbone.stem.0.bn.running_var: copying a param with shape torch.Size([16]) from checkpoint, the shape in current model is torch.Size([24]).
size mis

In [12]:
PreTrainDataset = How2Sign(video_dir="data/raw_videos",
                           MP_model=MP_model,
                           body_cfg=MMPoseCFG,
                           body_model=body_model)
def collate_as_is(batch):
    return batch
PreTrainDataLoader = DataLoader(PreTrainDataset, batch_size=4, shuffle=False, collate_fn=collate_as_is)

In [13]:
OneSample = next(iter(PreTrainDataLoader))

In [17]:
len(OneSample)

4